<a href="https://colab.research.google.com/github/pollyjuice74/REU-LDPC-Project/blob/main/LinearTranDiff_on_5G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Source code
!git clone https://github.com/pollyjuice74/REU-LDPC-Project
!git clone https://github.com/pollyjuice74/DDECC
# Sionna stuff
!git clone https://github.com/NVlabs/sionna.git
!pip install mitsuba
!pip install pythreejs

In [2]:
import tensorflow as tf
import random
import numpy as np
import scipy as sp
import time
import os
from scipy.sparse import issparse, csr_matrix, coo_matrix


# os.chdir('../..')
os.chdir('sionna')
from sionna.fec.ldpc.encoding import * # 5g encoder

from sionna.utils.metrics import compute_ber, compute_bler
from sionna.utils import BitErrorRate, BinarySource, ebnodb2no

from sionna.mapping import Constellation, Mapper, Demapper
from sionna.channel import AWGN
os.chdir('..')

if os.path.exists('REU-LDPC-Project'):
  os.rename('REU-LDPC-Project', 'REU_LDPC_Project')

os.chdir('REU_LDPC_Project/adv_nn')
from attention import *
from channel import *
from transformer import *
from dataset import *
from args import *
from model_functs import *
from models import *
from decoder5G import * # 5g decoder

os.chdir('../..')
from DDECC.src.codes import EbN0_to_std


In [3]:
class TransformerDiffusion( Layer ):
    def __init__(self, args):
        super().__init__()
        self.model_type = args.model_type
        self.n_steps = args.n_steps

        code = args.code
        assert issparse(code.H), "Code's pcm must be sparse."
        self.pcm = code.H
        # shapes
        self.m, self.n = self.pcm.shape
        self.k = self.n - self.m

        self.mask = self.create_mask(self.pcm)
        # layers
        self.src_embed = tf.Variable( tf.random.uniform([1, self.n + self.m, args.d_model]), trainable=True )
        self.decoder = Transformer(args.d_model, args.heads, self.mask, args.t_layers)
        self.fc = Dense(1)
        self.to_n = Dense(1)
        self.to_m = Dense(1)
        self.time_embed = Embedding(args.n_steps, args.d_model)

        self.betas = tf.constant( tf.linspace(1e-3, 1e-2, args.n_steps)*0 + args.sigma )
        self.betas_bar = tf.constant( tf.math.cumsum(self.betas, 0) )

        self.split_diff = False#args.split_diff
        self.ls_active = args.ls_active

        scheduler = tf.keras.optimizers.schedules.CosineDecay( initial_learning_rate=args.lr, decay_steps=args.epochs ) # 1000 is size of trainloader
        self.optimizer =  tf.keras.optimizers.Adam(learning_rate=scheduler)

    def create_mask(self, H):
        m,n = H.shape
        mask = tf.eye(n+m, dtype=tf.float32) # (n+m, n+m)
        cn_con, vn_con, _ = sp.sparse.find(H)

        for cn, vn_i in zip(cn_con, vn_con):
            # cn to vn connections in the mask
            mask = tf.tensor_scatter_nd_update(mask, [[n+cn, vn_i],[vn_i, n+cn]], [1.0,1.0])

            # distance 2 vn neighbors of vn_i
            related_vns = vn_con[cn_con==cn]
            for vn_j in related_vns:
                mask = tf.tensor_scatter_nd_update(mask, [[vn_i, vn_j],[vn_j, vn_i]], [1.0,1.0])

        # -infinity where mask is not set
        mask = tf.cast( tf.math.logical_not(mask > 0), dtype=tf.float32) # not(mask > 0) for setting non connections to -1e9
        return mask

    def get_sigma(self, t):
        # make sure t is a positive int
        t = tf.cast( tf.abs(t), tf.int32 )
        # gather betas
        betas_t = tf.gather(self.betas, t)
        betas_bar_t = tf.gather(self.betas_bar, t)

        return betas_bar_t * betas_t / (betas_bar_t + betas_t)

    def get_syndrome(self, r_t):
        # Calculate syndrome (pcm @ r = 0) if r is correct in binary
        r_t = tf.reshape(r_t, (self.n, -1)) # (n,b)
        return self.pcm.dot( llr_to_bin( r_t ).numpy() ) % 2 # (m,n)@(n,b)->(m,b)

    # Extracts noise estimate z_hat from r
    def tran_call(self, r_t, t):
        # Make sure r_t and t are compatible
        r_t = tf.reshape(r_t, (self.n, -1)) # (n,b)
        t = tf.cast(t, dtype=tf.int32)

        # Compute synd and magn
        syndrome = tf.reshape( self.get_syndrome(llr_to_bin(r_t)), (self.pcm.shape[0], -1) ) # (m,n)@(n,b)->(m,b) check nodes
        magnitude = tf.reshape( tf.abs(r_t), (self.n, -1) ) #(n,b) variable nodes
        # make sure their the same dtype
        magnitude, syndrome = [ tf.cast(tensor, dtype=tf.float32) for tensor in [magnitude, syndrome] ]

        # Concatenate synd and magn
        nodes = tf.concat([magnitude, syndrome], axis=0) # data for vertices
        nodes = tf.reshape(nodes, (1, self.n+self.m, -1)) # (1, n+m, b)
        print(nodes.shape)

        # Embedding nodes w/ attn and 'time' (sum syn errs) dims
        nodes_emb = tf.reshape( self.src_embed * nodes, (self.src_embed.shape[-1], self.pcm.shape[0]+self.n, -1) ) # (d,n+m,b)
        time_emb = tf.reshape( self.time_embed(t), (self.src_embed.shape[-1], 1, -1) ) # (d,1,b)

        # Applying embeds
        emb_t = time_emb * nodes_emb # (d, n+m, b)
        logits = self.decoder(emb_t) # (d, n+m, d) # TODO: missing batch dims b
        print(emb_t, logits)

        # Reduce (d,n+m,d)->(d,n+m)
        logits = tf.squeeze( self.fc(logits), axis=-1 )
        vn_logits = tf.reshape( logits[:, :self.n], (self.n, -1) ) # (n,d) take the first n logits from the concatenation
        cn_logits = tf.reshape( logits[:, self.n:], (self.m, -1) ) # (m,d) take the last m logits from the concatenation
        print(vn_logits, cn_logits)

        z_hat = self.to_n(vn_logits)# (n,d)->(n,)
        synd = self.to_m(cn_logits)# (m,d)->(m,)
        print(logits.shape, z_hat.shape)

        return z_hat, synd

    # optimal lambda l for theoretical and for error prediction
    def line_search(self, r_t, sigma, err_hat, lin_splits=20):
        l_values =  tf.reshape( tf.linspace(1., 20., lin_splits), (1, 1, lin_splits) )
        r_t, sigma, err_hat = [ tf.expand_dims(tensor, axis=-1) for tensor in [r_t, sigma, err_hat] ]# (n,b, 1)
        print(f"sigma: {sigma}, err_hat: {err_hat}")

        # Compute theoretical step size w/ ls splits
        z_hat_values = l_values*(sigma*err_hat) # (n,b, l), l is lin_splits
        r_values = llr_to_bin(r_t - z_hat_values) # (n,b, l)
        # sum of synds (m,n)@(n,b*l)->(m,b*l)->(b*l, 1)
        sum_synds = tf.reduce_sum( tf.abs( self.pcm.dot( tf.squeeze(r_values, axis=1) ) % 2 ),
                                   axis=0 )[:, tf.newaxis]
        print(sum_synds.shape)

        # Pick optimal ls value
        if self.model_type=='dis':
             ixs = tf.math.argmin(sum_synds, axis=0, output_type=tf.int32) # (b,1) w/ ixs of optimal line search for batch b
        elif self.model_type=='gen':
             ixs = tf.math.argmax(sum_synds, axis=0, output_type=tf.int32) # (b,1)

        print(r_values.shape, z_hat_values.shape, ixs.shape)
        # (b, l, n) for indexing on l
        r_values, z_hat_values = [ tf.transpose(tensor, perm=[1,2,0])
                                            for tensor in [r_values, z_hat_values] ]

        # concat range of batch ixs [0,...,n-1] and optimal line search ixs in gather_nd
        indices = tf.concat( [tf.range(ixs.shape[0]), ixs], axis=-1) # (b,2)

        # print(r_values, z_hat_values, indices)
        # ix on lin_splits w/ gather_nd st. ix,(b, l, n)->(n,b)
        r_t1, z_hat = [ tf.reshape( tf.gather_nd(tensor, indices), (self.n, -1) )
                                             for tensor in [r_values, z_hat_values] ]
        print(r_t1, z_hat_values)
        return r_t1, z_hat # r at t-1

    def loss_fn(self, synd):
        return tf.reduce_mean(tf.square(synd))

    def train_step(self, llr_ch):
        with tf.GradientTape() as tape:
            _, synd = self.tran_call(llr_ch,
                                     tf.reduce_sum( self.get_syndrome(llr_ch) ))
            loss = self.loss_fn(synd)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss



class Decoder( TransformerDiffusion ):
    def __init__(self, args):
        super().__init__(args)

    # 'test' function
    def call(self, r_t):
        for i in range(self.m):
            print(r_t.shape)
            # both (n,)
            r_t, z_hat = self.rev_diff_call(r_t) if not self.split_diff else self.split_rdiff_call(r_t)

            # Check if synd is 0 return r_t
            if tf.reduce_sum( self.get_syndrome(r_t) ) == 0:
                return r_t, z_hat, i

        return r_t, z_hat, i

    # Refines recieved codeword r at time t
    def rev_diff_call(self, r_t):
        print("Rev def call...")
        # Make sure r_t and t are compatible
        r_t = tf.reshape(r_t, (self.n, -1)) # (n,b)
        # 'time step' of diffusion is really ix of abs(sum synd errors)
        t = tf.reduce_sum( self.get_syndrome(llr_to_bin(r_t)), axis=0 ) # (m,n)@(n,b)->(m,b)->(1,b)
        t = tf.cast(tf.abs(t), dtype=tf.int32)

        # Transformer error prediction
        z_hat_crude, synd = self.tran_call(r_t, t) # (n,1), (m,1)
        print("z_hat_crude: ", z_hat_crude)

        # Compute diffusion vars
        sigma = self.get_sigma(t) # theoretical step size
        print("sigma: ", sigma)
        err_hat = r_t - tf.sign(z_hat_crude * r_t) # (n,1)

        # Refined estimate of the codeword for the ls diffusion step
        r_t1, z_hat = self.line_search(r_t, sigma, err_hat) if self.ls_active else 1.
        # r_t1[t==0] = r_t[t==0] # if cw has 0 synd. keep as is

        return r_t1, z_hat # r at t-1, both (n,1)

    def split_rdiff_call(self, r_t):
        print("Rev diff call with split diffusion...")
        # Ensure r_t is correctly shaped
        r_t = tf.reshape(r_t, (self.n, -1))  # (n,b)
        t = tf.reduce_sum(self.get_syndrome(llr_to_bin(r_t)), axis=0)  # (m,n)@(n,b)->(m,b)->(1,b)
        t = tf.cast(tf.abs(t), dtype=tf.int32)

        # First half-step condition subproblem
        z_hat_crude, synd = self.tran_call(r_t, t)
        r_t_half = r_t - 0.5 * self.fc(z_hat_crude * self.get_sigma(t))

        # Full-step diffusion subproblem
        r_t1 = r_t_half + tf.random.normal(r_t_half.shape) * tf.sqrt(self.get_sigma(t))

        # Second half-step condition subproblem
        z_hat_crude_half, synd = self.tran_call(r_t1, t)  # Reuse the second `tran_call`
        r_t1 = r_t1 - 0.5 * self.fc(z_hat_crude_half * self.get_sigma(t))

        return r_t1, z_hat_crude_half  # r at t-1, both (n,1)



args = Args(model_type='dis', code_type='LDPC5G') # args for decoder/discriminator

# Define enc/dec layers #
enc5G = LDPC5GEncoder(args.k, args.n)
dec5G = LDPC5GDecoder(enc5G, args)

binary_source = BinarySource()

# initialize mapper and demapper for constellation object
constellation = Constellation("qam", num_bits_per_symbol=4)
mapper = Mapper(constellation=constellation)
demapper = Demapper("app", constellation=constellation)

channel = AWGN() # replace w/ Generator(args)

no = ebnodb2no(1, 11, args.k/args.n) # eb_no=1, bps=4
no = tf.expand_dims(tf.cast(no, tf.float32), axis=-1)

args.code.H = dec5G.pcm
dec = Decoder(args)



# Simulate #



In [7]:
dec5G = LDPC5GDecoder(enc5G, args)
# dec = Decoder(args)

def train_dec_5G(dec5G, epoch, training_len=100):
    loss_fn = tf.keras.losses.BinaryCrossentropy()
    t = time.time()

    for batch_idx in range(training_len):
        u = binary_source([1, args.k])
        c = enc5G(u) # (1,n)

        x = mapper(c) # map c to symbols x
        y = channel([x, no]) # transmit over AWGN channel
        llr_ch = demapper([y, no]) # demap y to LLRs (1,n)

        if dec5G.return_llrs5g:
            llr_5g = dec5G(llr_ch)

            loss = dec.train_step(llr_5g)

        else:
            c_hat = dec5G(llr_ch)
            print("c, c_hat: ", c, c_hat)
            loss = loss_fn(c_hat, c)

        if (batch_idx + 1) % 1 == 0:
            print(f'Training epoch {epoch}, Batch {batch_idx + 1}/{training_len}, Loss={loss.numpy():.5e}')

    print(f'Epoch {epoch} Train Time {time.time() - t}s\n')


train_dec_5G(dec5G, epoch=1)

(1, 764, 1)
tf.Tensor(
[[[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 ...

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]], shape=(128, 764, 1), dtype=float32) tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0.

Training epoch 1, Batch 1/100, Loss=2.85477e+00
(1, 764, 1)
tf.Tensor(
[[[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 ...

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]], shape=(128, 764, 1), dtype=float32) tf.Tensor(
[[[-0.10004186 -0.20162575 -0.14838    ...  0.42604607  0.08091199
   -0.03277602]
  [-0.10004186 -0.20162575 -0.14838    ...  0.42604607  0.08091199
   -0.03277602]
  [-0.10004186 -0.20162575 -0.14838    ...  0.42604607  0.08091199
   -0.03277602]
  ...
  [-0.10004186 -0.20162575 -0.14838    ...  0.42604607  0.08091199
   -0.03277602]
  [-0.10004186 -0.20162575 -0.14838    ...  0.42604607  0.08091199
   -0.03277602]
  [-0.10004186 -0.20162575 -0.14838    ...  0.42604607  0.08091199
   -0.03277602]]

 [[-0.10004186 -0.20162575 -0.148

Training epoch 1, Batch 2/100, Loss=3.49990e+01
(1, 764, 1)
tf.Tensor(
[[[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 ...

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]], shape=(128, 764, 1), dtype=float32) tf.Tensor(
[[[ 0.06061488 -0.03742179 -0.08985044 ...  0.18859294  0.05053362
   -0.04918952]
  [ 0.06061488 -0.03742179 -0.08985044 ...  0.18859294  0.05053362
   -0.04918952]
  [ 0.06061488 -0.03742179 -0.08985044 ...  0.18859294  0.05053362
   -0.04918952]
  ...
  [ 0.06061488 -0.03742179 -0.08985044 ...  0.18859294  0.05053362
   -0.04918952]
  [ 0.06061488 -0.03742179 -0.08985044 ...  0.18859294  0.05053362
   -0.04918952]
  [ 0.06061488 -0.03742179 -0.08985044 ...  0.18859294  0.05053362
   -0.04918952]]

 [[ 0.06061488 -0.03742179 -0.089

Training epoch 1, Batch 3/100, Loss=4.82552e+00
(1, 764, 1)
tf.Tensor(
[[[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 ...

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]], shape=(128, 764, 1), dtype=float32) tf.Tensor(
[[[ 0.16521327  0.10187042 -0.05762851 ...  0.00711305  0.07028772
   -0.07937425]
  [ 0.16521327  0.10187042 -0.05762851 ...  0.00711305  0.07028772
   -0.07937425]
  [ 0.16521327  0.10187042 -0.05762851 ...  0.00711305  0.07028772
   -0.07937425]
  ...
  [ 0.16521327  0.10187042 -0.05762851 ...  0.00711305  0.07028772
   -0.07937425]
  [ 0.16521327  0.10187042 -0.05762851 ...  0.00711305  0.07028772
   -0.07937425]
  [ 0.16521327  0.10187042 -0.05762851 ...  0.00711305  0.07028772
   -0.07937425]]

 [[ 0.16521327  0.10187042 -0.057

Training epoch 1, Batch 4/100, Loss=5.29308e+00
(1, 764, 1)
tf.Tensor(
[[[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 ...

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]], shape=(128, 764, 1), dtype=float32) tf.Tensor(
[[[ 0.15200846  0.16585082 -0.07513744 ... -0.08671951  0.09484892
   -0.14707254]
  [ 0.15200846  0.16585082 -0.07513744 ... -0.08671951  0.09484892
   -0.14707254]
  [ 0.15200846  0.16585082 -0.07513744 ... -0.08671951  0.09484892
   -0.14707254]
  ...
  [ 0.15200846  0.16585082 -0.07513744 ... -0.08671951  0.09484892
   -0.14707254]
  [ 0.15200846  0.16585082 -0.07513744 ... -0.08671951  0.09484892
   -0.14707254]
  [ 0.15200846  0.16585082 -0.07513744 ... -0.08671951  0.09484892
   -0.14707254]]

 [[ 0.15200846  0.16585082 -0.075

...

Training epoch 1, Batch 96/100, Loss=3.68598e-04
(1, 764, 1)
tf.Tensor(
[[[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 ...

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]], shape=(128, 764, 1), dtype=float32) tf.Tensor(
[[[-0.37243855  0.34051955 -0.08935297 ... -0.22403358  0.19550994
   -0.23061465]
  [-0.37243855  0.34051955 -0.08935297 ... -0.22403358  0.19550994
   -0.23061465]
  [-0.37243855  0.34051955 -0.08935297 ... -0.22403358  0.19550994
   -0.23061465]
  ...
  [-0.37243855  0.34051955 -0.08935297 ... -0.22403358  0.19550994
   -0.23061465]
  [-0.37243855  0.34051955 -0.08935297 ... -0.22403358  0.19550994
   -0.23061465]
  [-0.37243855  0.34051955 -0.08935297 ... -0.22403358  0.19550994
   -0.23061465]]

 [[-0.37243855  0.34051955 -0.08

Training epoch 1, Batch 97/100, Loss=2.13889e-04
(1, 764, 1)
tf.Tensor(
[[[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 ...

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]], shape=(128, 764, 1), dtype=float32) tf.Tensor(
[[[-0.37326583  0.3406416  -0.09002879 ... -0.22357787  0.19612816
   -0.23112273]
  [-0.37326583  0.3406416  -0.09002879 ... -0.22357787  0.19612816
   -0.23112273]
  [-0.37326583  0.3406416  -0.09002879 ... -0.22357787  0.19612816
   -0.23112273]
  ...
  [-0.37326583  0.3406416  -0.09002879 ... -0.22357787  0.19612816
   -0.23112273]
  [-0.37326583  0.3406416  -0.09002879 ... -0.22357787  0.19612816
   -0.23112273]
  [-0.37326583  0.3406416  -0.09002879 ... -0.22357787  0.19612816
   -0.23112273]]

 [[-0.37326583  0.3406416  -0.09

Training epoch 1, Batch 98/100, Loss=1.84018e-04
(1, 764, 1)
tf.Tensor(
[[[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 ...

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]], shape=(128, 764, 1), dtype=float32) tf.Tensor(
[[[-0.37411618  0.34076694 -0.09073316 ... -0.22309943  0.19677027
   -0.23165336]
  [-0.37411618  0.34076694 -0.09073316 ... -0.22309943  0.19677027
   -0.23165336]
  [-0.37411618  0.34076694 -0.09073316 ... -0.22309943  0.19677027
   -0.23165336]
  ...
  [-0.37411618  0.34076694 -0.09073316 ... -0.22309943  0.19677027
   -0.23165336]
  [-0.37411618  0.34076694 -0.09073316 ... -0.22309943  0.19677027
   -0.23165336]
  [-0.37411618  0.34076694 -0.09073316 ... -0.22309943  0.19677027
   -0.23165336]]

 [[-0.37411618  0.34076694 -0.09

Training epoch 1, Batch 99/100, Loss=1.50720e-04
(1, 764, 1)
tf.Tensor(
[[[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 0.]
  [ 0.]
  [ 0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 ...

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]

 [[-0.]
  [-0.]
  [-0.]
  ...
  [-0.]
  [-0.]
  [-0.]]], shape=(128, 764, 1), dtype=float32) tf.Tensor(
[[[-0.37484917  0.3408751  -0.09134496 ... -0.22268628  0.19732493
   -0.23211044]
  [-0.37484917  0.3408751  -0.09134496 ... -0.22268628  0.19732493
   -0.23211044]
  [-0.37484917  0.3408751  -0.09134496 ... -0.22268628  0.19732493
   -0.23211044]
  ...
  [-0.37484917  0.3408751  -0.09134496 ... -0.22268628  0.19732493
   -0.23211044]
  [-0.37484917  0.3408751  -0.09134496 ... -0.22268628  0.19732493
   -0.23211044]
  [-0.37484917  0.3408751  -0.09134496 ... -0.22268628  0.19732493
   -0.23211044]]

 [[-0.37484917  0.3408751  -0.09

Training epoch 1, Batch 100/100, Loss=2.18612e-04
Epoch 1 Train Time 1836.7272102832794s



In [ ]:
cn_con, vn_con, vals = sp.sparse.find(dec5G.pcm)
cn_con, vn_con

H = [[1,0,1,1,1,0,0],
     [0,1,0,1,1,1,0],
     [0,0,1,0,1,1,1]]
H = csr_matrix(H)

def get_syndrome(pcm, r_t):
        # Calculate syndrome (pcm @ r = 0) if r is correct in binary
        r_t = tf.reshape(r_t, (pcm.shape[1], -1)) # (n,b)
        return pcm.dot(llr_to_bin(r_t).numpy()) % 2

get_syndrome(dec5G.pcm, llr_5g)

In [ ]:
## DATA ###
EbNo_range_train = range(2, 8)
EbNo_range_test = range(5, 14)
# Standard deviation for train/test
std_train = [EbN0_to_std(ii, args.k / args.n) for ii in EbNo_range_train]
std_test = [EbN0_to_std(ii, args.k / args.n) for ii in EbNo_range_test]

scheduler = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate=args.lr, decay_steps=args.epochs) # 1000 is size of trainloader
optimizer = tf.keras.optimizers.Adam(learning_rate=scheduler)

train_loader = FEC_Dataset(args.code, sigma=std_test, zero_cw=True, length=args.traindata_len).batch(args.batch_size).shuffle(buffer_size=args.batch_size)

#                         z_cw   m 1s   1-cw     Should use zero codeword by default
dataset_types = {
              "bin_bits":(False, False, False), # Binary bits sent and recieved with some awgn
              "flip_cw": (True, False, True),   # Zero codeword flipped to a all ones vector [1,1,...,1]
              "zero_cw": (True, False, False),  # Standard zero codeword used for training
              "ones_m":  (False, True, False),  # Makes the message all ones vector and passes it to generator matrix producing codeword and pcm
              }

test_ebnos_datasets = [ [FEC_Dataset(args.code, sigma=std_test, zero_cw=zero_cw, ones_m=ones_m, flip_cw=flip_cw)
                                 for ii in range(len(std_test))]
                                 for (zero_cw, ones_m, flip_cw) in dataset_types.values() ]

In [ ]:

for epoch in range(1, args.epochs + 1):
    print("Training Linear Transformer Diffusion Model...")
    # train_dec(dec, train_loader, optimizer, epoch,
    #           LR=scheduler(tf.Variable(0, dtype=tf.float32)).numpy(),
    #           traindata_len=args.traindata_len)

    # print comparison
    if epoch % 1 == 0:
        data = test_models(dec, test_ebnos_datasets, EbNo_range_test)
    break # from for loop

data

In [ ]:
import matplotlib.pyplot as plt
def plot_comparison(title, EbNo_range, ber, fer, diff_iters):
    plt.figure(figsize=(12, 6))

    # Plot diff_iters to decoding
    plt.subplot(1, 2, 1)
    plt.plot(EbNo_range, diff_iters, marker='o', label='diff_iters to decode', color='blue')
    plt.yscale('log')
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('diff_iters to decode')
    plt.title('diff_iters to decode vs Eb/No')
    plt.grid(True, which="both", ls="--")
    plt.legend()

    # Plot BER for both models
    plt.subplot(1, 2, 2)
    plt.plot(EbNo_range, ber, marker='x', label='BER', color='green')
    plt.yscale('log')
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('BER')
    plt.title('BER vs Eb/No')
    plt.grid(True, which="both", ls="--")
    plt.legend()

    # Plot FER for both models
    plt.subplot(1, 2, 3)
    plt.plot(EbNo_range, fer, marker='o', label='FER', color='red')
    plt.yscale('log')
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('FER')
    plt.title('FER vs Eb/No')
    plt.grid(True, which="both", ls="--")
    plt.legend()

    # Set the overall title for the figure
    plt.suptitle(title)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


for ix, dataset_type in enumerate(dataset_types.keys()):
    ber = data['LTDM'][ix]['ber']
    fer = data['LTDM'][ix]['bler']
    diff_iters = data['LTDM'][ix]['diff_iters']

    plot_comparison(dataset_type.upper(), EbNo_range_test, ber, fer, diff_iters)